## Project Proposal

Name: Gabriella Martinez

Topic: Displacement in Puerto Rico

Data: The World Bank

### Part 1

My project is inspired by rising concerns of gentrification and displacement of local populations in Puerto Rico. I have heard conflicting news regarding the state of Puerto Rican culture and livlihood due to the influx of Americans and other foriengers to the island. From this interest, I decided to study population trends in Puerto Rico, to uncover the truth about what is happening on the island. I aim to answer the following questions: How are economic factors influincing migration to or from Puerto Rico for natives and foriegners? How has gentrification impacted housing affordability and displacement in Puerto Rico?

### Part 2

In [5]:
import requests
import json
import pandas as pd

In [6]:
# PR Urban Population Growth
url1 = 'https://api.worldbank.org/V2/country/PRI/indicator/SP.URB.TOTL.IN.ZS?format=json'
response1 = requests.get(url1)
PR_pop_growth = response1.json()
print(PR_pop_growth[1])

# PR GDP per Capita
url2 = 'https://api.worldbank.org/V2/country/PRI/indicator/NY.GDP.PCAP.CD?format=json'
response2 = requests.get(url2)
PR_gdp = response2.json()
PR_gdp
print(PR_gdp[1])

# PR Migration Data
url3 = 'https://api.worldbank.org/V2/country/PRI/indicator/SM.POP.NETM?format=json'
response3 = requests.get(url3)
PR_migration = response3.json()
PR_migration[1]

[{'indicator': {'id': 'SP.URB.TOTL.IN.ZS', 'value': 'Urban population (% of total population)'}, 'country': {'id': 'PR', 'value': 'Puerto Rico'}, 'countryiso3code': 'PRI', 'date': '2023', 'value': 93.634, 'unit': '', 'obs_status': '', 'decimal': 0}, {'indicator': {'id': 'SP.URB.TOTL.IN.ZS', 'value': 'Urban population (% of total population)'}, 'country': {'id': 'PR', 'value': 'Puerto Rico'}, 'countryiso3code': 'PRI', 'date': '2022', 'value': 93.61, 'unit': '', 'obs_status': '', 'decimal': 0}, {'indicator': {'id': 'SP.URB.TOTL.IN.ZS', 'value': 'Urban population (% of total population)'}, 'country': {'id': 'PR', 'value': 'Puerto Rico'}, 'countryiso3code': 'PRI', 'date': '2021', 'value': 93.592, 'unit': '', 'obs_status': '', 'decimal': 0}, {'indicator': {'id': 'SP.URB.TOTL.IN.ZS', 'value': 'Urban population (% of total population)'}, 'country': {'id': 'PR', 'value': 'Puerto Rico'}, 'countryiso3code': 'PRI', 'date': '2020', 'value': 93.581, 'unit': '', 'obs_status': '', 'decimal': 0}, {'in

[{'indicator': {'id': 'SM.POP.NETM', 'value': 'Net migration'},
  'country': {'id': 'PR', 'value': 'Puerto Rico'},
  'countryiso3code': 'PRI',
  'date': '2023',
  'value': 19833,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SM.POP.NETM', 'value': 'Net migration'},
  'country': {'id': 'PR', 'value': 'Puerto Rico'},
  'countryiso3code': 'PRI',
  'date': '2022',
  'value': 16704,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SM.POP.NETM', 'value': 'Net migration'},
  'country': {'id': 'PR', 'value': 'Puerto Rico'},
  'countryiso3code': 'PRI',
  'date': '2021',
  'value': -7410,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SM.POP.NETM', 'value': 'Net migration'},
  'country': {'id': 'PR', 'value': 'Puerto Rico'},
  'countryiso3code': 'PRI',
  'date': '2020',
  'value': -7410,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SM.POP.NETM', 'value': 'Net migration'},
  'country': {'i

In [7]:
# make the population growth dataframe
pop_growth_df = pd.json_normalize(PR_pop_growth[1])
pop_growth_df = pop_growth_df[['date', 'value']]
pop_growth_df = pop_growth_df.rename(columns={'value': 'Urban Population Growth'})
pop_growth_df.head()

,date,Urban Population Growth
0,2023,93.634
1,2022,93.610
2,2021,93.592
3,2020,93.581
4,2019,93.576


In [8]:
# make the GDP per capita dataframe
gdp_df = pd.json_normalize(PR_gdp[1])
gdp_df = gdp_df[['date', 'value']]
gdp_df = gdp_df.rename(columns={'value': 'GDP Per Capita'})
gdp_df.head()

,date,GDP Per Capita
0,2023,36779.059491
1,2022,35268.079102
2,2021,32619.250417
3,2020,31427.429114
4,2019,32916.866801


In [9]:
# make the migration dataframe
migration_df = pd.json_normalize(PR_migration[1])
migration_df = migration_df[['date', 'value']]
migration_df = migration_df.rename(columns={'value': 'Migration'})
migration_df.head()

,date,Migration
0,2023,19833
1,2022,16704
2,2021,-7410
3,2020,-7410
4,2019,10623


In [10]:
# merge the dataframes
merged_df = pd.merge(pop_growth_df, gdp_df, on='date')
PR_df = pd.merge(merged_df, migration_df, on='date')
PR_df.head()

,date,Urban Population Growth,GDP Per Capita,Migration
0,2023,93.634,36779.059491,19833
1,2022,93.610,35268.079102,16704
2,2021,93.592,32619.250417,-7410
3,2020,93.581,31427.429114,-7410
4,2019,93.576,32916.866801,10623


In [34]:
# make a categorical column for decade
decades = []
for value in PR_df['date']:
    decade = value[0:3] + '0'
    decades.append(decade)

PR_df['decade'] = decades
PR_df.head()

,date,Urban Population Growth,GDP Per Capita,Migration,decade
0,2023,93.634,36779.059491,19833,2020
1,2022,93.610,35268.079102,16704,2020
2,2021,93.592,32619.250417,-7410,2020
3,2020,93.581,31427.429114,-7410,2020
4,2019,93.576,32916.866801,10623,2010


### Visualizations

In [4]:
import matplotlib.pyplot as plt

### Part 3

The data I have chosen will help answer my research questions by providing insight into different aspects of Puerto Rico’s economic and demographic changes. The World Bank offers multiple datasets that allow for an in-depth analysis of key factors related to gentrification and population shifts. From this, I will examine GDP per capita, urban population growth, and migration data. Analyzing GDP per capita over time will help determine how Puerto Rico’s economic landscape has changed, identifying periods of significant growth or decline. Urban population growth data will allow me to see how cities are expanding and whether urbanization trends align with economic changes. Migration data will provide insights into population movements—whether people are leaving or relocating to specific areas—and how those shifts may correlate with economic factors. Each dataset includes 'date' and 'value' attributes, which I will use to merge the datasets and create a clean, unified dataset. This will allow me to track trends over time and analyze relationships between GDP, migration, and urban population growth. By examining these variables together, I aim to uncover patterns that may indicate economic or demographic shifts contributing to gentrification in Puerto Rico.